In [ ]:
# from datasets import load_dataset, DatasetDict
# Assuming you have a dataset in CSV format
# climate_change_dataset = load_dataset("csv", data_files={"train": "path/to/train.csv", "validation": "path/to/val.csv"})
# Example structure of each data point: {'query': 'What is climate change?', 'document': 'Climate change refers to...', 'answer': 'Climate change is...'}

In [5]:
from transformers import RagTokenizer, RagTokenForGeneration
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: a2506e9a-cd0c-43d3-862f-5966b66f2547)')

In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./rag_finetuned/",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs/",
    logging_steps=10,
)
# Define a function to process the data for training
def preprocess_function(examples):
    inputs = [ex["query"] + tokenizer.sep_token + ex["document"] for ex in examples]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    # Prepare labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["answer"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Process the dataset
tokenized_datasets = climate_change_dataset.map(preprocess_function, batched=True)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)
# Start fine-tuning
trainer.train()

In [ ]:
from transformers import RagTokenizer, RagTokenForGeneration
# Load the fine-tuned model and tokenizer
model = RagTokenForGeneration.from_pretrained("./rag_finetuned/")
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
# Generating content
query = "Explain the impact of global warming on polar ice caps."
input_ids = tokenizer(query, return_tensors="pt").input_ids
# Generate the answer
generated_ids = model.generate(input_ids)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

In [3]:
import pandas as pd
from transformers import RagTokenizer, RagTokenForGeneration

# Read data from Excel file
data = pd.read_csv("../deskapp/email_results.csv")


OSError: Incorrect path_or_model_id: './rag_finetuned/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.